In [11]:
%load_ext autoreload
%autoreload 2
from configs import project_config
import matplotlib.pyplot as plt
import numpy as np
from configs.RecursiveVPSDE.LSTM_fMullerBrown.recursive_LSTM_PostMeanScore_MullerBrown_T256_H05_tl_110data import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
config = get_config()

In [13]:
def rmse_ignore_nans(y_true, y_pred):
    assert (y_true.shape == y_pred.shape and len(y_pred.shape) == 1)
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)  # Ignore NaNs in both arrays
    return np.sqrt(np.mean((y_true[mask] - y_pred[mask]) ** 2))

In [14]:
ts_type = "MullerBrown"
include = (f"{config.t0}t0_{config.deltaT:.3e}dT").replace(".", "")

In [15]:
import os
keepfiles = []
root_dir = project_config.ROOT_DIR + f"experiments/results/"
for file in os.listdir(root_dir):
    if "Nadaraya" in file and ts_type in file and include in file and "DriftEvalExp" in file:
        keepfiles.append(root_dir+file)
print(len(keepfiles), ts_type)
assert len(keepfiles)>0, "No files found"
def extract_bw_number(s):
    match = s.split("/")[-1].split("_")[3][:-2]
    return float("0." + match)

keepfiles = sorted(keepfiles, key=extract_bw_number)
Xshape = np.load(keepfiles[0]).shape[0]
bws = [extract_bw_number(f) for f in keepfiles]

0 MullerBrown


AssertionError: No files found

In [ ]:
def find_true_drift(prev, numXs, config):
    assert (prev.shape == (numXs, config.ndims))
    Aks = np.array(config.Aks)[np.newaxis, :]
    aks = np.array(config.aks)[np.newaxis, :]
    bks = np.array(config.bks)[np.newaxis, :]
    cks = np.array(config.cks)[np.newaxis, :]
    X0s = np.array(config.X0s)[np.newaxis, :]
    Y0s = np.array(config.Y0s)[np.newaxis, :]
    common = Aks * np.exp(aks* np.power(prev[:,[0]] - X0s, 2) \
                                 + bks* (prev[:,[0]] - X0s) * (prev[:, [1]] - Y0s)
                                 + cks* np.power(prev[:, [1]] - Y0s, 2))
    assert (common.shape == (numXs, 4))
    drift_X = np.zeros((numXs, config.ndims))
    drift_X[:, 0] = -np.sum(common * (2. * aks* (prev[:, [0]] - X0s) + bks* (prev[:, [1]] - Y0s)), axis=1)
    drift_X[:, 1] = -np.sum(common * (2. * cks* (prev[:, [1]] - Y0s) + bks* (prev[:, [0]] - X0s)), axis=1)

    return drift_X

In [ ]:
numXs = 25
minx = -1.
maxx = -0.9
Xs = np.linspace(minx, maxx, numXs)
miny = 1.
maxy = 1.1
Ys = np.linspace(miny, maxy, numXs)
X, Y = np.meshgrid(Xs, Ys)
Xs = np.column_stack([X.ravel(), Y.ravel()])
num_paths = 10952
is_true_drifts = find_true_drift(prev=Xs, numXs=Xshape, config=config)
print(ts_type)

In [ ]:
def plot_drift_estimator(mean, true_drift, coords, ts_type, bw, toSave:bool = False):
    fig, ax = plt.subplots(figsize=(20,12))
    rmse = rmse_ignore_nans(true_drift, mean).astype(np.float64)
    error_data = np.power(np.mean(np.power(true_drift  - mean,2), axis=-1), 0.5)
    assert (error_data.shape == (true_drift.shape[0],))

    # Extract x and y from coordinates
    x, y = coords[:, 0], coords[:, 1]

    # Create a heatmap using tricontourf
    plt.tricontourf(x, y, error_data, levels=20, cmap="hot")  # Contour plot with 20 levels
    cbar = plt.colorbar(label="RMSE Value")  # Add color bar
    cbar.ax.tick_params(labelsize=38)  # Change font size of numbers on colorbar
    cbar.set_label("RMSE Value", fontsize=38)
    ax.set_title(rf"RMSE {round(rmse,3)} for LSTM Score Estimator", fontsize=40)
    ax.tick_params(labelsize=38)
    ax.set_xlabel("State $X_{1}$", fontsize=38)
    ax.set_ylabel("State $X_{2}$", fontsize=38)
    ax.legend(fontsize=24)
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
best_rmse = np.inf
toSave = False
for fidx in keepfiles:
    f = keepfiles[fidx]
    bw = bws[fidx]
    try:
        print(f)
        is_drift = np.load(f).mean(axis=1)
        assert (is_drift.shape == Xs.shape)
        fig, ax = plt.subplots(figsize=(14,9))
        rmse = rmse_ignore_nans(is_true_drifts, is_drift).astype(np.float64)#np.power(np.mean(np.power(true_drift - mean, 2)), 0.5)
        if rmse < np.inf*best_rmse:
            best_rmse = rmse
            plot_drift_estimator(mean=is_drift, toSave=toSave, true_drift=is_true_drifts, ts_type=ts_type, bw=bw, coords=Xs)
        print(f"Bandwidth {bw} done with Best RMSE {best_rmse}\n\n\n\n")
    except ValueError as e:
        plt.close()
        continue
